# Optimizing Online 5g Machine-Learning with Nsight Compute

## 07 Balance memory and computation

After the previous optimization to move the basis vectors into shared memory for better re-use, we were still facing `Barrier` warps stalls.

In the previous version, `all loading from global memory into shared memory was done by a single warp` at the beginning of the loop. During this time, `the rest of the warps are idle (stalled)`, wasting precious cycles waiting at a `barrier`. This can be improved in two ways: we can read smaller batches and alternate them with compute; and we can `perform more efficient reads by issuing more accesses in parallel`. For the latter, we use the warps of other samples. In this way, we aim at balancing the memory and compute operations. Another consequence of this implementation is that we no longer limit the length of the input to a max vector length, as the algorithm is now independent of vector sizes.

You know the drill: set the `APSM_DETECT_VERSION` flag in [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h) to `apsm_version::APSM_DETECT_BALANCED` and re-compile the code with the following command and collect the report:

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!make -j
!ncu -k kernel_apsm_detect --set full --import-source yes -f -o /dli/task/ncu/report_balanced \
    bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

One last time, open the new report file `/root/Desktop/reports/ncu/report_balanced.ncu-rep` in the Nsight Compute UI.

Again, our optimization had a very positive effect on the kernel runtime, reducing it by 60%. In addition, compute and memory utilization are now very well balanced and both around 67%. The tool considers the kernel not latency-bound anymore at this point,

<img src="images/ncu_report04_01.png">

The same latency reduction is also visible in the `Scheduler Statistics` section, where `Issued Warp Per Scheduler` improved 76% and is now at a decent ~37%. Improvements to this metric regularly have a very positive effect on kernel performance.

<img src="images/ncu_report04_02.png">

Finally, the `Warp States` chart highlights that `Barrier stalls have become completely negligible` (while `MIO Throttle` stalls have increased).

Of course, further optimizations would be possible. MIO Throttle indicates that we are still accessing memory too aggressively. Upon close inspection, we can see that there are massive shared memory bank conflicts (check the Memory Workload Analysis section: the vector sizes cause collisions. To mitigate this, we can add padding, which should reduce the number of bank conflicts.

In fact, in a real-world scenario, our optimization goal would be a runtime of less than 1 ms on the target hardware. However, we will consider the kernel now fast enough for the purpose of this training.

You are now ready to move to the [summary](08_summary.ipynb) of this Nsight Compute tutorial.